In [2]:
import sys
import os
import glob
import json
import pandas as pd
%matplotlib inline
import matplotlib
import numpy as np

In [3]:
def find_files(path):
    return glob.glob(path)

In [4]:
def read_json(filename):
    with open(filename) as json_file:
        data = json.load(json_file)
    return data

## Adatok olvasása
Adatok beolvasása JSON fájlként, majd egy nagy pandas-os DataFrame-mé alakítása.

In [5]:
json_data = []

for file in find_files('Data/*'):
    print("Reading file:", file)
    json_data.append(read_json(file))
    
data_frame = pd.DataFrame()

for data in json_data:
    frames  = [data_frame, pd.DataFrame(data)]
    data_frame = pd.concat(frames).reset_index(drop=True)

Reading file: Data/gyakori_szorakozas_30000
Reading file: Data/gyakori_allatok_14000
Reading file: Data/gyakori_egeszseg
Reading file: Data/gyakori_egeszseg_20000
Reading file: Data/gyakori_szamitastechnika


## Bepillantás a kérdésekbe
Csak hogy tudjuk pontosan mivel is állunk szemben. Minden kérdéshez tartozik a kérdés rövid, illetve hosszú verziója, egy válasz, amit a felhasználók a leghasznosabbnak találtak. Ezeken kívül kategóriák és kulcsszavak is vannak a kérdéshez.

In [6]:
data_frame.head(2)

,kategoriak,hosszu_kerdes,rovid_kerdes,keywords,valasz
0,"[Szórakozás, Filmek, sorozatok]",Milyen animék vannak amikben a tanulás szerepe...,Milyen animék vannak amikben a tanulás szerepe...,"[rész, tanulás, iskola, anime]",Baka To TestAnsatsu KyoushitsuBokutachi wa Ben...
1,"[Szórakozás, Filmek, sorozatok]",Nem látta már valaki véletlen ezt a filmet? Ez...,Nem látta már valaki véletlen ezt a filmet? Ez...,"[Tumblr, film, gif, mozgókép, csók]","Nagyon ügyes a barátod, én is szoktam vele bes..."


A kategóriák közül csak a "főkategória" lényeges számunka, ezért a többit elhagyjuk.

In [7]:
data_frame['kategoriak'] = data_frame['kategoriak'].apply(lambda x: x[0])

## Átlagos hossz, extrém esetek szűrése
Az extrém hosszú vagy rövid kérdések nem túl hasznosak. Érdemes őket kiszűrni. Az átlagostól sokkal eltérő kérdéseket nem használjuk a továbbiakban.

In [8]:
def calculate_avg(data_frame, label="rovid_kerdes"):
    l = data_frame[label].tolist()
    return sum(map(len, l)) / len(l)

def drop_extreme(data_frame, min_, max_, label="rovid_kerdes"):
    data_frame = data_frame[data_frame[label].map(len) >= min_]
    data_frame = data_frame[data_frame[label].map(len) <= max_]

    return data_frame

In [9]:
long_questions_avg_len = calculate_avg(data_frame, "hosszu_kerdes")
print("Average length: %f" % long_questions_avg_len)

Average length: 346.495816


In [10]:
data_frame = drop_extreme(data_frame, long_questions_avg_len / 4, long_questions_avg_len * 3, "hosszu_kerdes")
long_questions_avg_len = calculate_avg(data_frame, "hosszu_kerdes")

print("Average length: %f" % long_questions_avg_len)

Average length: 326.753950


## "Főkategóriák" kigyűjtése
Az egyes főkategóriák neveinek kigyűjtése, majd az egyes nevekhez egy azonosító szám rendelése.

In [11]:
target_names = data_frame['kategoriak'].unique().tolist()
target_dict =  {value: key for key, value in enumerate(target_names)}

num_of_categories = len(target_names)

print(target_names)
print(target_dict)

['Szórakozás', 'Állatok', 'Egészség', 'Számítástechnika']
{'Szórakozás': 0, 'Állatok': 1, 'Egészség': 2, 'Számítástechnika': 3}


## Tanító adathalmaz előállítása
A tanító adathalmazban minden egyes főkategóriából ugyanannyi kérdésnek kell szerepelnie (így fair). Itt pontosan ez történik `questions_from_each_category` darab kérdés kerül a tanító adathalmazba minden kategóriából, majd az eredményül kapott tömb véletlenszerűen összekeveredik.

#### Shuffle together
A shuffle_together függvény két listát véletlenszerűen kever össze, úgy, hogy az a keverés előtt az egyes listákban azonos indexen szereplő értékek a keverés után is azonos indexen lesznek.

In [12]:
def shuffle_together(list1, list2):
    zipped = list(zip(list1, list2))
    random.shuffle(zipped)
    list1, list2 = zip(*zipped)
    
    return (list(list1), list(list2))

In [15]:
import random

train_each_ctg = 2100
train_size = train_each_ctg * num_of_categories
train_questions = []
train_target = []

test_each_ctg = 1200
test_size = test_each_ctg * num_of_categories
test_questions = []
test_target = []

# Shuffle rows
data_frame = data_frame.sample(frac=1).reset_index(drop=True)

for target_name in target_names:
    train_questions += data_frame[data_frame["kategoriak"] == target_name][0:train_each_ctg]["hosszu_kerdes"].to_list()
    train_target += [target_dict[target_name]] * train_each_ctg    
    
    test_questions += data_frame[data_frame["kategoriak"] == target_name][train_each_ctg:train_each_ctg + test_each_ctg]["hosszu_kerdes"].to_list()
    test_target += [target_dict[target_name]] * test_each_ctg    

train_questions, train_target = shuffle_together(train_questions, train_target)
test_questions, test_target = shuffle_together(test_questions, test_target)

In [16]:
def unicode_to_ascii(data):
    return unidecode.unidecode(re.sub(r"[,.;@#?!&$]+\ *", " ", data).lower()).split()

In [17]:
import unidecode
import re
from collections import Counter

vocab = set()

for idx, q in enumerate(train_questions):
    words = unicode_to_ascii(q)

    for word in words:
        vocab.add(word)

    train_questions[idx] = " ".join(words)

total_words = len(vocab)

index_of_word = {}

for idx, word in enumerate(vocab):
    index_of_word[word.lower()] = idx

In [18]:
def get_batch(text, target=[0], i=0, batch_size=1):

    batches = []
    results = []
    
    texts = text[i * batch_size : i * batch_size + batch_size]
    categories = target[i * batch_size : i * batch_size + batch_size]

    for text in texts:
        layer = np.zeros(input_size , dtype=float)
        for word in text.split():
            if word.lower() in index_of_word:
                layer[index_of_word[word.lower()]] += 1
            
        batches.append(layer)
        
    for category in categories:
        results.append(category)
     
    return np.array(batches), np.array(results)

In [19]:
learning_rate = 0.01
num_epochs = 5
batch_size = 150

hidden_size = 100
input_size = total_words
num_classes = len(target_names)

In [20]:
from torch.autograd import Variable
import torch.nn as nn
import torch

In [21]:
class Classification(nn.Module):
     def __init__(self, input_size, hidden_size, num_classes):
        super(Classification, self).__init__()
        self.layer_1 = nn.Linear(input_size, hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)
 
     def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [22]:
net = Classification(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  


for epoch in range(num_epochs):
    if epoch:
        print()
    print("Epoch %d/%d: " % (epoch + 1, num_epochs))
    total_batch = len(train_questions) // batch_size
    for i in range(total_batch):
        batch_x, batch_y = get_batch(train_questions, train_target, i, batch_size)
        questions = Variable(torch.FloatTensor(batch_x))
        themes = Variable(torch.LongTensor(batch_y))
        
        optimizer.zero_grad()
        outputs = net(questions)
        loss = criterion(outputs, themes)
        loss.backward()
        optimizer.step()
        
        print("\r[%d/%d] %.2f%%" % (i + 1, total_batch, (i + 1)/ total_batch * 100), end="")

Epoch 1/5: 


/home/versenyi/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


[56/56] 100.00%
Epoch 2/5: 
[56/56] 100.00%
Epoch 3/5: 
[56/56] 100.00%
Epoch 4/5: 
[56/56] 100.00%
Epoch 5/5: 
[56/56] 100.00%

In [48]:
total_batch = len(test_questions) // batch_size

total = 0
correct = 0

for i in range(total_test_batch):
    test_batch_x, test_batch_y = get_batch(test_questions, test_target, i, batch_size)
    print("\rTesting... [%d/%d] %.2f%%" % (i + 1, total_batch, (i + 1)/ total_batch * 100), end="")
    
    questions = Variable(torch.FloatTensor(test_batch_x))
    themes = Variable(torch.FloatTensor(test_batch_y))
    
    outputs = net(questions)
    _, predicted = torch.max(outputs.data, 1)
    
    correct += (predicted == themes).sum().item()
    total += len(themes)

print("\n\nResults:\n%d test\n%d correct prediciton\n%d wrong prediction\n%.2f accuracy" % (total, correct, total - correct, 100 ))

Testing... [32/32] 100.00%
Results:
4800 test
3527 correct prediciton
1273wrong prediction



In [22]:
print("Mennyi kérdésed van?")
q_num = int(input())

for q in range(q_num):
    test_text = input()

    test_data = unicode_to_ascii(test_text)

    batch_x_test, _ = get_batch(test_data)
    question = Variable(torch.FloatTensor(batch_x_test))
    outputs = net(question)
    _, predicted = torch.max(outputs.data, 1)

    print("A kérdés %s témájú .. talán." % target_names[predicted.item()])

Mennyi kérdésed van?
0
